https://kontur.renote.team/doc/LMnJ0F18P


Существует сервис Omnidesk, это тикетная система. Тут можно прочитать про их API - https://omnidesk.ru/api/introduction/intro

В ней есть несколько сущностей, но нас интересует сущность Обращений (https://omnidesk.ru/api/cases).

Нужно написать ETL скрипт на Python, который будет обращаться к API Omnidesk и выгружать списком все обращения, оставленные после заданной даты. Сценарий использования — ежемесячная аналитика успеваемости техподдержки.

Запишите результат в любую SQL базу на ваше усмотрение. 

Также ждем небольшое пояснение о том, почему приняли именно такое решение и какие видите у проблемные моменты.

Ждем jupyter-notebook, ссылку на git или python файл с решением. 

Нас интересует не столько результат задания, сколько ваше умение организовывать код, так чтобы с ним было приятно работать и легко поддерживать. Уделите этому внимание, пожалуйста

P.S. Вы можете зарегистрировать бесплатный тестовый аккаунт на омни, чтобы создавать и редактировать обращения.


Для первоначальной загрузки данных в данной ситуации можно использовать nosql базу, или как вариант сделать тыблицы: Обращение, Почта(или юзер, тут надо по смыслу данных смотреть), и таблицы связи обащения и почты для тех полей, которые апи возвращает массивами, но все же мне кажется удобнее и логичнее для сырых данных использовать nosql, вроде mongodb или elastic, возможно hadoop

In [77]:
import requests
import json
import uuid
import psycopg2
import sqlite3

In [15]:
api_key = '***'
domain = '***'
mail = '***'
from_time = '2022-01-25'

In [47]:
def request(body, mail, api_key):
    response = requests.get(body, headers=headers, auth=(mail, api_key))
    if response.status_code == 200:
        return response.json()
    return {}

def get_cases_body(domain, limit=100, page=0, from_time=''):
    body = f'https://{domain}.omnidesk.ru/api/cases.json?limit={limit}&page={page}'
    
    if from_time:
        body += f'&from_time={from_time}'

    headers = {
        'Content-Type': 'application/json',
    }

    return body

def load(domain, mail, api_key, from_time=''):
    mas = []
    f_end_page = True
    limit=100
    page=0
    
    if f_end_page:
        body = get_cases_body(domain, limit=limit, page=page, from_time=from_time)
        r = request(body, mail, api_key)
        
        total_count = r.pop('total_count')
        if limit*(page+1) > total_count:
            f_end_page = False
        
        for case in r.values():
            d = case['case']
            mas.append(d)
    
    return mas

In [73]:
def save_data(data, name):
    with open(name, 'w') as file:
        json.dump(data, file)

def open_data(name):
    with open(name, 'r') as file:
        data = json.load(file)
    return data

In [143]:
def transform_row(row):
    for key in row.keys():
        if type(row[key]) is list:
            row[key] = f"'{json.dumps(row[key])}'"
        elif type(row[key]) is str:
            row[key] = f"'{row[key]}'"
        elif type(row[key]) is int:
            row[key] = f"{row[key]}"
        elif type(row[key]) is bool:
            row[key] = f"{row[key]}"
    return row

def transform_column(row):
    body = []
    for key in row.keys():
        if type(row[key]) is list or type(row[key]) is str:
            body.append(f"{key} text")
        elif type(row[key]) is int:
            body.append(f"{key} int")
        elif type(row[key]) is bool:
            body.append(f"{key} bool")
    return body

In [88]:
class DB_sqlite3:
    def __init__(self, db):
        self.conn = sqlite3.connect(db)        
        self.cursor = self.conn.cursor()
    
    def execute(self, body):
        self.cursor.execute(body)
        self.conn.commit()
        
    def select(self, body):
        self.cursor.execute(body)
        return self.cursor.fetchall()
    
    def close(self):
        self.conn.close()
        
    def __del__(self):
        self.conn.close()

In [148]:
def send_data(data):
    columns = ', '.join(transform_column(data[0]))
    body = f"""CREATE TABLE IF NOT EXISTS cases
            (
                {columns}
            )
    """  
    db = DB_sqlite3('my_base.db')
    db.execute(body)
    
    for row in data:
        d = transform_row(row)
        values = ', '.join(d.values())
        req = f"""INSERT INTO cases VALUES ({values});"""
        db.execute(req)

In [118]:
data = load(domain, mail, api_key, from_time=from_time)
name = f"{domain}__{uuid.uuid4()}.json"
save_data(data, name)

In [149]:
data = open_data(name)
send_data(data)

In [146]:
# DB_sqlite3('my_base.db').select('select * from cases;')

[(196896327,
  '276-780550',
  'Наполнить базу знаний полезными статьями',
  31408347,
  36111,
  70885,
  'open',
  'low',
  'idea',
  'support@omnidesk.ru',
  '',
  '',
  '["support@omnidesk.ru"]',
  '[]',
  '[]',
  0,
  0,
  'Tue, 25 Jan 2022 20:38:56 +0300',
  '-',
  'Tue, 25 Jan 2022 20:38:38 +0300',
  'Tue, 25 Jan 2022 20:38:39 +0300',
  0,
  '-',
  1,
  '[83373]'),
 (196896325,
  '817-782685',
  'Подключить Facebook-страницу к центру поддержки',
  31408347,
  36111,
  70885,
  'open',
  'low',
  'idea',
  'support@omnidesk.ru',
  '',
  '',
  '["support@omnidesk.ru"]',
  '[]',
  '[]',
  0,
  0,
  'Tue, 25 Jan 2022 20:38:55 +0300',
  '-',
  'Tue, 25 Jan 2022 20:38:38 +0300',
  'Tue, 25 Jan 2022 20:38:38 +0300',
  0,
  '-',
  1,
  '[83373]'),
 (196896324,
  '130-561186',
  'Подключить Twitter-аккаунт к центру поддержки',
  31408347,
  36111,
  70885,
  'open',
  'low',
  'idea',
  'support@omnidesk.ru',
  '',
  '',
  '["support@omnidesk.ru"]',
  '[]',
  '[]',
  0,
  0,
  'Tue, 25 J